# Computational Methods in Astrophysics: Assignment 3
##### Prasad Rajesh Posture
Roll No.: MSC2303121013

# Importing Dependencies

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
from IPython import display
%matplotlib tk

#### Total Variation Diminishing Schemes
**Total Variation:** It is a measure of how much a function oscillates or varies within a given interval. It is given as $$TV(q) = \sum_{i=0}^{N} |q_{i} - q_{i-1}|$$
**TVD Scheme:**
A numerical scheme is said to be total variation diminishing (TVD) scheme if
$$ TV(q^{n+1}) \leq TV(q^n)$$

Such a scheme will not develop oscillations near a jump, because a jump is a monotonically
in/decreasing function and a total variation diminishing scheme will not increase the TV ,
and hence in this case keep the TV identically constant. This appears to be a desirable property
of the numerical scheme.

#### Slope Limiters
Sharp changes or discontinuities in a solution can introduce errors in numerical simulations because traditional numerical methods, like finite difference or finite volume schemes, struggle to accurately represent and resolve abrupt changes in the solution. When a sharp change occurs, the solution gradients become very steep, leading to large numerical errors and oscillations in the computed solution. These errors arise because the numerical method may not capture the rapid changes adequately, leading to inaccuracies in the computed values. **Slope limiters** help mitigate these issues by smoothing out the solution near sharp changes, thereby reducing the steepness of the gradients and suppressing oscillations. They restrict the steepness of the solution gradients to avoid unphysical behavior like overshoots and oscillations, especially in regions of sharp gradients or discontinuities.

#### Flux Limiters
Flux limiters are primarily used to control the propagation of information across grid cells in hyperbolic PDEs. They limit the magnitude of the numerical fluxes to prevent the formation of spurious oscillations and ensure stability and accuracy of the solution, particularly in regions of steep gradients or discontinuities. Flux limiters are essential for capturing shock waves and other discontinuities accurately, as they help maintain the monotonicity and positivity of the solution by controlling the fluxes at cell interfaces.

#### Difference between Slope Limiter and Flux Limiter
1) Slope limiters are used to control the steepness of solution gradients within individual grid cells. Flux limiters, on the other hand, are used to control the propagation of information across grid cells in hyperbolic.<br>
2) PDEs.Slope Limiters: Applied within individual grid cells to control the rate of change of the solution. Flux Limiters: Applied at cell interfaces to control the magnitude of fluxes between adjacent grid cells.<br>
3) Slope Limiters: Smoothen out the solution gradients within cells, preventing unphysical oscillations near sharp changes. Flux Limiters: Control the propagation of information between grid cells, ensuring stability and accuracy of the solution by limiting the magnitude of fluxes, especially in regions of steep gradients or discontinuities.

#### Defining all the required functions

In [2]:
# Defining Phi's: Flux Limiters
# Defining flux limiter functions
# works
def donner_cell(r):
    "Donner Cell"
    return 0.0
# works
def lax_wendroff(r):
    "Lax-Wendroff"
    return 1.0

# works
def beam_warming(r):
    "Beam-Warming"
    return r

# works
def fromm(r):
    "Fromm"
    return (1/2)*(1+r)

# works
def minmod(r):
    "Minmod"
    if r>0:
        if 1<=abs(r):
            return 1
        elif 1>abs(r):
            return r
        else:
            pass
    else:
        return 0

# works
def superbee(r):
    "Superbee"
    return max(0, min(1,2*r), min(2,r))

# No reference to verify
# It is some similiar to superbee, given that the superbee isnt working properly
# Same issue could be with this one
def mc(r):
    "MC"
    return max(0, min((1+r)/2, 2, 2*r))

# no reference 
# lets assume that it works
def van_leer(r):
    "van Leer"
    return (r+abs(r))/(1+abs(r))


In [3]:
# Defining a solver function

def pde_solver(method,dt = 10**(-3), c = 3.0, nt=10):
    global method_, dt_, nt_
    method_ = method
    nx = 200
    x = np.linspace(-10, 10, nx)
    dx = x[1] - x[0]
    dt_ = dt    
    nt_ = nt                                          
    t = np.arange(0,nt, dt)          
    pulse_height = 1.0
    pulse_width = 5.0
    q0 = np.where((np.abs(x) <= pulse_width), pulse_height, 0)
    
    q = q0.copy()
    sol = []
   
    for i in range(len(t)):
        qn = q.copy()
        sol.append(qn)
       
        for i in range(nx):
            theta = 1
        
            dm = ((qn[i] - qn[(i-1) % nx]))
            if dm != 0:
                rnm = (qn[(i - 1) % nx] - qn[(i - 2) % nx]) / dm
            else:
                rnm = 0.0
            phim = method(rnm)
            
            dp = ((qn[(i+1)%nx] - qn[i]))
            if dp !=0:
                rnp = (qn[i] - qn[(i - 1) % nx]) / dp
            else:
                rnp = 0.0
            phip = method(rnp)

            
            fnm = c/2 * ((1 + theta)*qn[(i - 1) % nx] + (1 - theta)*qn[i]) + abs(c)/2 * (1-abs(c*dt/dx))*(qn[i] - qn[(i - 1) % nx]) * phim
            fnp = c/2 * ((1 + theta)*qn[i] + (1 - theta)*qn[(i + 1) % nx]) + abs(c)/2 * (1-abs(c*dt/dx))*(qn[(i + 1) % nx] - qn[i]) * phip
            
            q[i] = qn[i] + (dt/dx) * (fnm-fnp)
    return sol

In [4]:
# Defininga a plotter function
def animator(sol):
    x = np.linspace(-10,10,len(sol[0]))
    fig = plt.figure()
    lines = plt.plot([])
    line = lines[0]
    plt.xlim(-10,10)
    plt.ylim(-0.05, max(sol[0])+0.1)
    plt.ylabel("Amplitude")
    plt.xlabel("Position(x)")
    plt.title(method_.__doc__)
    def animate(frame):
        y = sol[frame*int((nt_/dt_)/(40*nt_))]
        line.set_data((x,y))
    anim = FuncAnimation(fig, animate, frames=40*nt_, interval=25)
    return anim, plt.show()

#### Running the simulation for each method

In [157]:
animator(pde_solver(donner_cell))

(<matplotlib.animation.FuncAnimation at 0x167158922d0>, None)

In [158]:
animator(pde_solver(lax_wendroff))

(<matplotlib.animation.FuncAnimation at 0x1671cf07920>, None)

In [7]:
animator(pde_solver(beam_warming))

(<matplotlib.animation.FuncAnimation at 0x1c07a88b380>, None)

In [8]:
animator(pde_solver(fromm))

(<matplotlib.animation.FuncAnimation at 0x1c07a89b050>, None)

In [9]:
animator(pde_solver(minmod))

(<matplotlib.animation.FuncAnimation at 0x1c07befd010>, None)

In [159]:
animator(pde_solver(superbee))

(<matplotlib.animation.FuncAnimation at 0x1671e48cd10>, None)

In [7]:
animator(pde_solver(mc))

(<matplotlib.animation.FuncAnimation at 0x1b7f4f18050>, None)

In [6]:
animator(pde_solver(van_leer))

(<matplotlib.animation.FuncAnimation at 0x1fd5546ee40>, None)

# 2. 1D Isothermal Shock Tube


#### A. 1D isothermal equations in their conservative forms
1. **Conservation of mass:**
   $$\frac{\partial \rho}{\partial t} + \frac{\partial (\rho v)}{\partial x} = 0$$

2. **Conservation of momentum:**
   $$\frac{\partial (\rho v)}{\partial t} + \frac{\partial (\rho v^2)}{\partial x} = -\frac{\partial P}{\partial x}$$

3. **Conservation of energy:**
   $$\frac{\partial (\rho e_{total})}{\partial t} + \frac{\partial (\rho v e_{total})}{\partial x} = -\frac{\partial (Pv)}{\partial x}$$

where:
- $\rho$ is the density of the fluid,
- $v$ is the velocity of the fluid in the x-direction,
- $P$ is the pressure of the fluid, given by $P = \rho c_0^2$
- $c_0$ is the Isothermal Sound Speed
- $e_{total}$ is the total energy per unit volume, which includes both internal energy and kinetic energy and is given by $e_{total} = e + \frac{1}{2} v^2$,
- $e$ is the internal/thermal energy per unit volume, given as $e = (\gamma -1) \frac{P}{\rho}$

#### B. CFL Condition
Given CFL Factor = 0.5, signal speed = $c_0=1$ and $\Delta x = 1$ with grid points = 100 and duration 30sec<br>
$\therefore$ using CFL Condition we have,<br>
$c_0 \frac{\Delta t}{\Delta x} \leq 0.5$<br>
Substituing the values of $\Delta x$ and $c_0$ for $\Delta t$, we have,<br>
$\Delta t = 0.5$

#### C. Solving the continuity equation using flux limiter : Superbee
$$\frac{\partial \rho}{\partial t} + \frac{\partial (\rho v)}{\partial x} = 0$$
Given, $$\rho(x, t=0) = 2\Theta(50-x) + 1$$
$$v(x, t=0) = 0$$
With closed (reflective) boundaries.<br>
Let $\rho=q_1$ and $\rho v = q_1v = f_1$  <br>
$$\frac{\partial q_1}{\partial t} + \frac{\partial f_1}{\partial x} = 0$$
$$q_{1,i+1}^n = q_{1,i+}^n - \frac{\Delta t}{\Delta x}[f_{1,i+\frac{1}{2}}^n - f_{1,i-\frac{1}{2}}^n]$$


In [45]:
dx = 1
x = np.arange(0,100,dx)
nx = len(x)
dt = 0.01
t = np.arange(0,30,dt)
nt = len(t)
c0 = 1.0

# pulse_height = 1.0
# pulse_width = 5.0
# q10 = np.where((np.abs(x) <= pulse_width), pulse_height, 0)

# Initial Conditions
q10 = 2*(np.where(50-x>0,1,0))+1
q20 = np.zeros_like(q10)

# Solution at full step
q1 = q10.copy()
q2 = q20.copy()

# Solution at half step
q1_h = q10.copy()
q2_h = q20.copy()

# 2D Solution Array
sol_q1 = []
sol_q2 = []


for n in range(nt):
    q1n = q1.copy()
    q2n = q2.copy()
    sol_q1.append(q1n)
    sol_q2.append(q2n)

    for i in range(nx):


        if (q1n[i]==0) or (q1n[(i+1)%nx]==0):
            u_hp, f1p, f2p = 0, 0, 0
        else:
            u_hp = 0.5*((q2n[i]/q1n[i]) + (q2n[(i+1)%nx]/q1n[(i+1)%nx]))
            if u_hp>0:
                f1p = q1n[i]*u_hp
                f2p = q2n[i]*u_hp
            elif u_hp<0:
                f1p = q1n[(i+1)%nx]*u_hp
                f2p = q2n[(i+1)%nx]*u_hp
            else:
                f1p = 0
                f2p = 0
        
        if (q1n[(i-1)%nx]==0) or (q1n[i]==0):
            u_hm, f1m, f2m = 0, 0, 0
        else:
            u_hm = 0.5*((q2n[(i-1)%nx]/q1n[(i-1)%nx]) + (q2n[i]/q1n[i]))

            if u_hm>0:
                f1m = q1n[(i-1)%nx]*u_hm
                f2m = q2n[(i-1)%nx]*u_hm
            elif u_hm<0:
                f1m = q1n[i]*u_hm
                f2m = q2n[i]*u_hm
            else:
                f1m = 0
                f2m = 0

        # Calculating at half step
        q1_h[i] = q1n[i] - (dt/dx)*(f1p - f1m)
        q2_h[i] = q2n[i] - (dt/dx)*(f2p - f2m)
    for j in range(nx):
        # Updating the source term
        q1[j] = q1_h[j]
        q2[j] = q2_h[j] - (1/dx)*((q1_h[(j+1)%nx]-q1_h[(j-1)%nx])*c0**2)

In [46]:
# Defininga a plotter function
def advection_plotter(sol, title):
    x = np.arange(-50,50,1)
    fig = plt.figure()
    lines = plt.plot([])
    line = lines[0]
    plt.xlim(-60,60)
    plt.ylim(-1, 3.5)
    plt.ylabel("Amplitude")
    plt.xlabel("Position(x)")
    plt.title(title)
    def animate(frame):
        y = sol[frame]
        line.set_data((x,y))
    anim = FuncAnimation(fig, animate, frames=len(sol), interval=50)
    return anim, plt.show()
advection_plotter(sol_q1,"1D Isothermal Shock Tube")

(<matplotlib.animation.FuncAnimation at 0x2a5a2216240>, None)

In [44]:
dx = 1
x = np.arange(0,100,dx)
nx = len(x)
dt = 0.5
t = np.arange(0,30,dt)
nt = len(t)
c0 = 1.0

pulse_height = 1.0
pulse_width = 5.0
# q10 = np.where((np.abs(x) <= pulse_width), pulse_height, 0)
q10 = 2*(np.where(50-x>0,1,0))+1
q20 = np.zeros_like(q10)

q1 = q10.copy()
q2 = q20.copy()

q1_h = q10.copy()
q2_h = q20.copy()

sol_q1 = []
sol_q2 = []


for n in range(nt):
    q1n = q1.copy()
    q2n = q2.copy()
    sol_q1.append(q1n)
    sol_q2.append(q2n)

    for i in range(0,nx-1):


        if (q1n[i]==0) or (q1n[(i+1)]==0):
            u_hp, f1p, f2p = 0, 0, 0
        else:
            u_hp = 0.5*((q2n[i]/q1n[i]) + (q2n[(i+1)]/q1n[(i+1)]))
            if u_hp>0:
                f1p = q1n[i]*u_hp
                f2p = q2n[i]*u_hp
            elif u_hp<0:
                f1p = q1n[(i+1)]*u_hp
                f2p = q2n[(i+1)]*u_hp
            else:
                f1p = 0
                f2p = 0
        
        if (q1n[(i-1)%nx]==0) or (q1n[i]==0):
            u_hm, f1m, f2m = 0, 0, 0
        else:
            u_hm = 0.5*((q2n[(i-1)]/q1n[(i-1)]) + (q2n[i]/q1n[i]))

            if u_hm>0:
                f1m = q1n[(i-1)]*u_hm
                f2m = q2n[(i-1)]*u_hm
            elif u_hm<0:
                f1m = q1n[i]*u_hm
                f2m = q2n[i]*u_hm
            else:
                f1m = 0
                f2m = 0

        # Calculating at half step
        q1_h[i] = q1n[i] - (dt/dx)*(f1p - f1m)
        q2_h[i] = q2n[i] - (dt/dx)*(f2p - f2m)
    for j in range(1,nx-1):
        # Updating the source term
        q1[j] = q1_h[j]
        q2[j] = q2_h[j] - (1/dx)*((q1_h[(j+1)]-q1_h[(j-1)])*c0**2)

advection_plotter(sol_q1,"1D Isothermal Shock Tube")

(<matplotlib.animation.FuncAnimation at 0x2a5a2488f50>, None)

In [6]:
def rho(nt):
    %matplotlib tk
    
    def heaviside(x):
        return np.where(x >= 0, 1, 0)
    nx = 100
    x = np.linspace(0, 100, nx)
    dx = x[1]-x[2]
    theta = heaviside(x)
    t = np.linspace(0,30,nt)
    dt = t[1] - t[0]
    # q1_0= 2 * theta * (50 - x) + 1
    q1_0 = 2*np.where(50-x>0, 1, 0) + 1
    v_0 = np.zeros(nx)
    q2_0 = q1_0*v_0
    
    fig, ax = plt.subplots()
    line = ax.plot(x, q1_0)[0]
    ax.set_xlabel("x")
    ax.set_ylabel("Function")
    ax.set_title(f"Fluxlimiter scheme")

    # Copying the initial conditions for the array size and later updating them based on the function
    q1 = q1_0.copy()
    q2 = q2_0.copy()
    v = v_0.copy()
    # Defining function that does the plotting
    def Algorithm(frame):
        q1n = q1.copy()
        q2n = q2.copy()
        vn = v.copy()
        # vn[0] = 0
        # vn[-1] = 0
        q1n[nx-1] = q1n[nx-2]
        q1n[0] = q1n[1]
        q2n[nx-1] = -q2n[nx-2]
        q2n[0] = -q2n[1]
        for i in range(1,nx-1):
            vn[i] = 0.5*((q2n[i]/q1n[i])+(q2n[i+1]/q1n[i+1]))   
            if vn[i] <  0:
                fp1 = vn[i]*q1n[(i+1)] 
                fp2 = vn[i]*q2n[(i+1)]
                fm1 = vn[i]*q1n[i]
                fm2 = vn[i]*q2n[i]
            else:
                fp1 = vn[i]*q1n[i]
                fp2 = vn[i]*q2n[i] 
                fm1 = vn[i]*q1n[(i-1)]
                fm2 = vn[i]*q2n[(i-1)] 

            q1[i] = q1n[i] - (dt / dx) * (fm1 - fp1)
            q2[i] = q2n[i] - (dt / dx) * (fm2 - fp2)
            q2[i] = q2[i] - (q1[i+1] - q1[i-1])/(2*dx)
          

        line.set_ydata(q1)
        return line

    # Create animation
    anim = FuncAnimation(fig, Algorithm, frames=nt, interval=50)

    # Show the animation
    return anim

In [7]:
rho(1000)

In [61]:
dx = 1
x = np.arange(0,100,dx)
nx = len(x)
dt = 0.05
t = np.arange(0,30,dt)
nt = len(t)
c0 = 1.0

pulse_height = 1.0
pulse_width = 5.0
# q10 = np.where((np.abs(x) <= pulse_width), pulse_height, 0)
q10 = 2*(np.where(50-x>0,1,0))+1
q20 = np.zeros_like(q10)

q1 = q10.copy()
q2 = q20.copy()

q1_h = q10.copy()
q2_h = q20.copy()

sol_q1 = []
sol_q2 = []


for n in range(nt):
    q1n = q1.copy()
    q2n = q2.copy()
    sol_q1.append(q1n)
    sol_q2.append(q2n)
    
    q1n[nx-1] = q1n[nx-2]
    q1n[0] = q1n[1]
    q2n[nx-1] = -q2n[nx-2]
    q2n[0] = -q2n[1]

    q1_h[nx-1] = q1_h[nx-2]
    q1_h[0] = q1_h[1]
    q2_h[nx-1] = q2_h[nx-2]
    q2_h[0] = q2_h[1]

    for i in range(1,nx-1):
        
        if (q1n[i]==0) or (q1n[i+1]==0):
            u_hp, f1p, f2p = 0, 0, 0
        else:
            u_hp = 0.5*((q2n[i]/q1n[i]) + (q2n[(i+1)]/q1n[(i+1)]))
            if u_hp>0:
                f1p = q1n[i]*u_hp
                f2p = q2n[i]*u_hp
            elif u_hp<0:
                f1p = q1n[(i+1)]*u_hp
                f2p = q2n[(i+1)]*u_hp
            else:
                f1p = 0
                f2p = 0
        
        if (q1n[(i-1)]==0) or (q1n[i]==0):
            u_hm, f1m, f2m = 0, 0, 0
        else:
            u_hm = 0.5*((q2n[(i-1)]/q1n[(i-1)]) + (q2n[i]/q1n[i]))

            if u_hm>0:
                f1m = q1n[(i-1)]*u_hm
                f2m = q2n[(i-1)]*u_hm
            elif u_hm<0:
                f1m = q1n[i]*u_hm
                f2m = q2n[i]*u_hm
            else:
                f1m = 0
                f2m = 0

        # Calculating at half step
        q1_h[i] = q1n[i] - (dt/dx)*(f1p - f1m)
        q2_h[i] = q2n[i] - (dt/dx)*(f2p - f2m)
    for j in range(1,nx-1):
        # Updating the source term
        q1[j] = q1_h[j]
        q2[j] = q2_h[j] - (1/dx)*((q1_h[(j+1)]-q1_h[(j-1)])*c0**2)

In [62]:
# Defininga a plotter function
def advection_plotter(sol, title):
    x = np.arange(-50,50,1)
    fig = plt.figure()
    lines = plt.plot([])
    line = lines[0]
    plt.xlim(-60,60)
    plt.ylim(-1, 3.5)
    plt.ylabel("Amplitude")
    plt.xlabel("Position(x)")
    plt.title(title)
    def animate(frame):
        y = sol[frame]
        line.set_data((x,y))
    anim = FuncAnimation(fig, animate, frames=len(sol), interval=300)
    return anim, plt.show()
advection_plotter(sol_q1,"1D Isothermal Shock Tube")

(<matplotlib.animation.FuncAnimation at 0x2a5986e81a0>, None)

In [10]:
def hydrodynamic_solver(method,dx=1,c0=1.0,dt=0.5):
    x = np.arange(0,100,dx)
    nx = len(x)
    t = np.arange(0, 30,dt)
    nt = len(t)

    # Initial Conditions
    q10 = 2*(np.where(50-x>0,1,0))+1
    q20 = np.zeros_like(q10)

    # Solution at full step
    q1 = q10.copy()
    q2 = q20.copy()

    # Solution at half step
    q1_h = q10.copy()
    q2_h = q20.copy()

    # 2D solution Array
    sol_q1 = []
    sol_q2 = []

    for i in range(nt):

        # Copies to be used later
        q1n = q1.copy()
        q2n = q2.copy()
        sol_q1.append(q1n)
        sol_q2.append(q2n)


        # Reflective boundary condition for full steps
        q1n[nx-1] = q1n[nx-2]
        q1n[0] = q1n[1]
        q2n[nx-1] = -q2n[nx-2]
        q2n[0] = -q2n[1]

        # # periodic boundary conditions
        # q1n[nx-1] = q1n[1]
        # q1n[0] = q1n[nx-2]
        # q2n[nx-1] = q2n[1]
        # q2n[0] = q2n[nx-2]

        # q1_h[nx-1] = q1_h[1]
        # q1_h[0] = q1_h[nx-2]
        # q2_h[nx-1] = q2_h[1]
        # q2_h[0] = q2_h[nx-2]

        # Reflective boundary conditions for half steps
        q1_h[nx-1] = q1_h[nx-2]
        q1_h[0] = q1_h[1]
        q2_h[nx-1] = -q2_h[nx-2]
        q2_h[0] = -q2_h[1]

        for i in range(1,nx-1):

            # Velocity for i+1/2
            if (q1n[i]==0) or (q1n[i+1]==0):
                u_hp, f1p, f2p = 0, 0, 0
            else:
                u_hp = 0.5*((q2n[i]/q1n[i]) + (q2n[(i+1)]/q1n[(i+1)]))
            
            # fluxes for i+1/2
            if u_hp>0:
                f1p = q1n[i]*u_hp
                f2p = q2n[i]*u_hp
            elif u_hp<0:
                f1p = q1n[(i+1)]*u_hp
                f2p = q2n[(i+1)]*u_hp
            else:
                f1p = 0
                f2p = 0

            # Velocity for i-1/2
            if (q1n[(i-1)]==0) or (q1n[i]==0):
                u_hm, f1m, f2m = 0, 0, 0
            else:
                u_hm = 0.5*((q2n[(i-1)]/q1n[(i-1)]) + (q2n[i]/q1n[i]))

            # fluxes for i-1/2
            if u_hm>0:
                f1m = q1n[(i-1)]*u_hm
                f2m = q2n[(i-1)]*u_hm
            elif u_hm<0:
                f1m = q1n[i]*u_hm
                f2m = q2n[i]*u_hm
            else:
                f1m = 0
                f2m = 0

            # Calculating at half step
            q1_h[i] = q1n[i] - (dt/dx)*(f1p - f1m)
            q2_h[i] = q2n[i] - (dt/dx)*(f2p - f2m)
        
        # Updating at full step
        for j in range(1,nx-1):
            q1[j] = q1_h[j]
            q2[j] = q2_h[j] - (1/dx)*((q1_h[(j+1)]-q1_h[(j-1)])*c0**2)

    return sol_q1

In [11]:
sw = hydrodynamic_solver("spring")

In [12]:
# Defininga a plotter function
def advection_plotter(sol, title):
    x = np.arange(-50,50,1)
    fig = plt.figure()
    lines = plt.plot([])
    line = lines[0]
    plt.xlim(-60,60)
    plt.ylim(-1, 3.5)
    plt.ylabel("Amplitude")
    plt.xlabel("Position(x)")
    plt.title(title)
    def animate(frame):
        y = sol[frame]
        line.set_data((x,y))
    anim = FuncAnimation(fig, animate, frames=len(sol), interval=300)
    return anim, plt.show()
advection_plotter(sw,"1D Isothermal Shock Tube")

(<matplotlib.animation.FuncAnimation at 0x260c6a377d0>, None)